In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


import time
from selenium.webdriver.common.keys import Keys
    
# data manipulate library
import pandas as pd

import csv

# address
# from geopy.geocoders import Nominatim

In [17]:
website = "https://app.joinhandshake.com/stu/postings"
path = "C:/Users/username/Downloads/chromedriver.exe"

driver = webdriver.Chrome()

driver.get(website)

In [18]:
# Xpath: Basic Job and Company Information
'''
Job Card = job-preview style__details___paBkq
Job_Title           = //*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]/h1
Job_Info            = //*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]/div[1]
Company_Name        = //*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/a
Company_Location    = //*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/div/div
Container           = //*[@id="skip-to-content"]/div[3]/div/div[1]/div/form/div[2]/div/div/div[2]/div[1]/div[1]
'''

# Class: Basic Job and Company Information
'''
Job_Title           = style__job-title___P7PJV
Job_Info            = style__job-type-info___MJ8oT
Company_Name        = style__employer-name___54lqg
Company_Location    = style__list-with-tooltip___Js1g4
'''

'\nJob_Title           = style__job-title___P7PJV\nJob_Info            = style__job-type-info___MJ8oT\nCompany_Name        = style__employer-name___54lqg\nCompany_Location    = style__list-with-tooltip___Js1g4\n'

In [19]:
# General Information
general_information_keys = ['Job Title', 'Job Info', 'Company Name', 'Company Location']

def get_general_information():
    job_title = driver.find_element(By.CLASS_NAME, 'style__job-title___P7PJV').text
    job_info = driver.find_element(By.CLASS_NAME, 'style__job-type-info___MJ8oT').text
    company_name = driver.find_element(By.CLASS_NAME, 'style__employer-name___54lqg').text
    company_location = driver.find_element(By.CLASS_NAME, 'style__list-with-tooltip___Js1g4').text
    return pd.Series([job_title, job_info, company_name, company_location], index=general_information_keys)

In [20]:
# About the Role
'''
class = style__col___5FTI6
title = style__title___5XGGK
content = style__content___w3TUd
'''

'\nclass = style__col___5FTI6\ntitle = style__title___5XGGK\ncontent = style__content___w3TUd\n'

In [21]:
# Card Title
def get_card_titles():
    card_titles = driver.find_elements(By.CLASS_NAME, "style__title___5XGGK")
    card_title = []
    for i in range(len(card_titles)):
        title = card_titles[i].text
        card_title.append(title)
    return card_title

In [22]:
# Card Content
def get_card_content():
    card_contents = driver.find_elements(By.CLASS_NAME, "style__content___w3TUd")
    card_content = []
    for i in range(len(card_contents)):
        content = card_contents[i].text
        card_content.append(content)
    return card_content

In [27]:
# Go to next page
def next_page():
    next_page_button = driver.find_element(By.XPATH, '//button[@aria-label="next page"]')
    next_page_button.click()
    time.sleep(5)

In [24]:
def print_case(number, df):
    width = 50
    print(('Case ' + str(number)).center(width, '-'))
    print(df)
    print('-'*width)

In [28]:
page = 1
count = 0
job_info = pd.DataFrame()

while True:

    if page == 3:
        break
    
    job_cards = driver.find_elements(By.XPATH, '//div/a[@data-hook="jobs-card"]')
    
    for job_card in job_cards:
        job_card.click()
        try:
            time.sleep(2) # wait for the job card to load
        except StaleElementReferenceException:
            time.sleep(5)

        # General Information
        try:
            job_info = get_general_information()
        except:
            print("Error: General Information")

        # About the Role: Title
        try:
            card_titles = get_card_titles()
        except:
            print("Error: Card Titles")

        # About the Role: Content
        try:
            card_contents = get_card_content()
        except:
            print("Error: Card Contents")

        about_the_role = pd.Series(card_contents, index=card_titles)
        df = pd.concat([job_info, about_the_role], axis=0)
        job_info = pd.concat([job_info, about_the_role], axis=0)

        count += 1

        # job_info = job_info.append(df, ignore_index=True)

        print_case(count, df)
        
    next_page()
    page += 1
    

----------------------Case 1----------------------
Job Title                Software Engineering Intern
Job Info                      Full-Time ∙ Internship
Company Name                                  Rapid7
Company Location                           Tampa, FL
Application deadline           May 31, 2023 12:00 AM
Posted date                         February 3, 2023
Location type                                On-site
US work authorization                       Required
dtype: object
--------------------------------------------------
----------------------Case 2----------------------
Job Title                           CRM / IT Intern
Job Info                     Part-Time ∙ Internship
Company Name             Star Mountain Capital LLC.
Company Location                          Tampa, FL
Application deadline         July 30, 2023 11:55 PM
Posted date                       February 13, 2023
Seasonal role                   (5/15/23 - 8/11/23)
Estimated pay                 $15.00-20.00 pe

In [29]:
print(job_info.head(10))

Job Title                Intern - Code Development for Scientific Compu...
Job Info                                            Full-Time ∙ Internship
Company Name                                  Sandia National Laboratories
Company Location                                             Livermore, CA
Application deadline                                March 17, 2023 2:59 AM
Posted date                                                  March 2, 2023
Estimated pay                                        $39.40-45.28 per hour
Location type                                                      On-site
US work authorization                                             Required
dtype: object
